In [ ]:
# Imports
from dotenv import load_dotenv
from glob import glob
from itertools import product
from json import dump as json_dump, load as json_load
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from tqdm import tqdm
import os
import polars as pl

In [ ]:
# Load environment variables
load_dotenv()

# Get the directory of the current file
__dir__ = Path(os.path.abspath(""))
"""
The directory of the current file
"""

# Load environment variables
DATASET_NAME = os.environ["DATASET_NAME"]
"""
Dataset name
"""

EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
"""
Embedding model name
"""

# Create the output directory
OUTPUT_DIRECTORY = __dir__ / f"../data/notebooks/parameter-search-embedding/{DATASET_NAME.replace("/", "-")}/{EMBEDDING_MODEL_NAME.replace("/", "-")}"
(OUTPUT_DIRECTORY / "results").mkdir(parents=True, exist_ok=True)

In [ ]:
# Load the embedded datasets
train_df = pl.read_parquet(__dir__ / f"../data/notebooks/classifier-embedding/{DATASET_NAME.replace("/", "-")}/{EMBEDDING_MODEL_NAME.replace("/", "-")}/train-embedded.parquet")
test_df = pl.read_parquet(__dir__ / f"../data/notebooks/classifier-embedding/{DATASET_NAME.replace("/", "-")}/{EMBEDDING_MODEL_NAME.replace("/", "-")}/test-embedded.parquet")

In [ ]:
def train_and_evaluate(ctx):
    """
    Train and evaluate the classifier for the given parameters
    """

    # Unpack the context
    index = ctx["index"]
    params = ctx["params"]

    try:
        with open(OUTPUT_DIRECTORY / f"results/{index}.json", "x") as result_file:
            # Train the classifier
            # See https://github.com/AhsanAyub/malicious-prompt-detection/blob/main/binary_classification.py
            classifier = RandomForestClassifier(criterion="gini", random_state=0, verbose=2, **params)
            classifier.fit(
                train_df["embeddings"].to_list(), 
                train_df["label"].to_list()
            )

            # Test the classifier
            y_predictions = classifier.predict(test_df["embeddings"].to_list())

            # Get the metrics
            y_actual = test_df["label"].to_list()

            auc = roc_auc_score(y_actual, y_predictions)
            report = classification_report(y_actual, y_predictions, target_names=["benign", "malicious"], output_dict=True)

            # Save the results
            result = {
                "index": index,
                "params": params,
                "auc": auc,
                "report": report,
            }

            json_dump(result, result_file, indent=4)
    except FileExistsError as err:
        print(f"Result for index {index} already exists ({err}). Skipping...")

In [ ]:
# Generate the contexts
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}
contexts = [
  {
    "index": i,
    "params": dict(zip(param_grid.keys(), values))
  } for i, values in enumerate(product(*param_grid.values()))
]

In [ ]:
# Grid search
for ctx in tqdm(contexts):
  train_and_evaluate(ctx)

In [ ]:
# Load the results
results = []
for filename in glob(str((OUTPUT_DIRECTORY / "results/*.json").resolve())):
    with open(filename, "r") as result_file:
        raw_result = json_load(result_file)
        result = {
            "index": raw_result["index"],
            "auc": raw_result["auc"],
            **({
                f"params_{k}": v
                for k, v in raw_result["params"].items()
            } | {
                f"report_{k}": v
                for k, v in raw_result["report"].items() if k in ["accuracy"]
            } | {
                f"report_{k1}_{k2}": v2
                for k1, v1 in raw_result["report"].items() if k1 in ["benign", "malicious", "macro avg", "weighted avg"]
                for k2, v2 in v1.items() if k2 in ["precision", "recall", "f1-score", "support"]
            })
        }
        results.append(result)

results_df = pl.DataFrame(results)

In [ ]:
ROWS = 9
COLS = 9
VISIBLE_PARAMETERS = {
    "n_estimators": "Number of Estimators",
    "max_depth": "Max Depth",
    "min_samples_split": "Min Samples Split",
    "min_samples_leaf": "Min Samples Leaf",
}
VISIBLE_REPORT_METRICS = {
    "auc": "AUC",
    "report_accuracy": "Accuracy",
    "report_weighted avg_precision": "Precision",
    "report_weighted avg_recall": "Recall",
    "report_weighted avg_f1-score": "F1 Score",
}
COLORS = ["blue", "green", "orange", "red", "purple"]

fig, axs = plt.subplots(ROWS, COLS, figsize=(20, 20))

# Plot the results
best_auc_index = results_df.sort("auc", descending=True)[0, "index"]
for row, col in product(range(ROWS), range(COLS)):
    index = row * COLS + col

    # Get the result
    result = results_df.filter(pl.col("index") == index).to_dicts()[0]

    # Plot the result (Make the title bold if result is best)
    axs[row, col].set_title(f"Result {index}", fontweight="bold" if index == best_auc_index else "normal")
    axs[row, col].bar(
        list(VISIBLE_REPORT_METRICS.values()),
        [result[k] for k in VISIBLE_REPORT_METRICS.keys()],
        color=COLORS,
    )
    axs[row, col].set_xticks([])
    axs[row, col].set_xlabel(
        f"({", ".join([
            str(result[f"params_{k}"] or "-")
            for k in VISIBLE_PARAMETERS.keys()
        ])})"
    )
    axs[row, col].set_ylim(0, 1)

# Title and labels
fig.suptitle("Random Forest Classifier Parameter Search Results", fontsize=16, y=1)
fig.supxlabel(f"Hyperparameters ({", ".join(VISIBLE_PARAMETERS.values())})", fontsize=12)

# Legend
handles = [
    plt.Rectangle((0, 0), 1, 1, color=color)
    for color in COLORS
]
labels = [
    *VISIBLE_REPORT_METRICS.values(),
]
fig.legend(handles, labels, loc="upper right")

plt.tight_layout()
plt.savefig(OUTPUT_DIRECTORY / "results.png", dpi=600)